In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

rng = np.random.default_rng()

# noise

#### Fourier decomp of time-varying matrix

In [ ]:
## fourier coeffs
C0, Cs1, Cc1, Cs2, Cc2 = [rng.normal(size=(2, 2)) for _ in range(5)]

In [ ]:
def C(t):
    """time-varying covariance matrix"""

    ## get angle
    theta = (t / 12) * 2 * np.pi

    ## eval sin/cos
    sin = np.sin(theta)
    cos = np.cos(theta)

    sin2 = np.sin(2 * theta)
    cos2 = np.cos(2 * theta)

    ## reconstruct
    return C0 + 0.5 * (sin * Cs1 + cos * Cc1) + 0.2 * (sin2 * Cs2 + cos2 * Cc2)


def C_eval(t_arr):
    """evaluate C at specified times"""
    return np.stack([C(t_) for t_ in t], axis=0)

In [ ]:
## eval. cyclostationary covariance
t = np.arange(12)
C_ = C_eval(t)

## do FFT
Chat = np.fft.fft(C_, axis=0)
omega = 2 * np.pi * np.fft.fftfreq(12)


def C_recon(t):
    return 1 / 12 * np.einsum("fij,f", Chat, np.exp(1j * omega * t)).real


def dCdt_recon(t):
    return 1 / 12 * np.einsum("fij,f", Chat, np.exp(1j * omega * t) * 1j * omega).real


print(np.allclose(C_recon(2.97), C(2.97)))

In [ ]:
## index to plot
j, i = (0, 0)

## compute matrices
t = np.linspace(0, 12)
dCdt_ = np.array([dCdt_recon(t_) for t_ in t])
C_ = np.array([C_recon(t_) for t_ in t])
dCdt_est = (C_[2:] - C_[:-2]) / (t[2] - t[0])

## plot results
fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(t, C_[:, j, i])
ax.plot(t, dCdt_[:, j, i])
ax.plot(t[1:-1], dCdt_est[:, j, i], ls="--")